In [1]:
import numpy as np
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn

In [2]:
data_path='./tap_fun_train.csv'
with open(data_path) as f:
    data=pd.read_csv(f)

In [3]:
x_train,x_test,y_train,y_test=train_test_split(data.iloc[:,2:-1],data['prediction_pay_price'],test_size=0.25)


In [9]:
pca=PCA(n_components=2)
pca.fit(data.iloc[:,2:-1])
pca_test=pca.transform(x_test)
pca_train=pca.transform(x_train)

In [10]:
dtrain=xgb.DMatrix(pca_train,label=y_train)
dtest=xgb.DMatrix(pca_test,label=y_test)

params = {

    'min_child_weight': 0.5,

    'eta': 0.03,

    'max_depth': 20,

    'subsample': 0.9,

    'alpha': 0.04,

    'gamma': 0,

    'silent': 0,

    'verbose_eval': True,

    'seed': 12

}

watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 500
bst = xgb.train(params, dtrain, num_round, watchlist)

bst.save_model('0001.model')

preds = bst.predict(dtest)
labels = dtest.get_label()

print('test rmse =%f' % (np.sqrt(sklearn.metrics.mean_squared_error(labels, preds))))


[0]	eval-rmse:100.963	train-rmse:82.1291
[1]	eval-rmse:99.9587	train-rmse:80.7067
[2]	eval-rmse:99.0014	train-rmse:79.3247
[3]	eval-rmse:98.1012	train-rmse:77.9154
[4]	eval-rmse:97.2302	train-rmse:76.5436
[5]	eval-rmse:96.3911	train-rmse:75.3405
[6]	eval-rmse:95.6149	train-rmse:74.2151
[7]	eval-rmse:94.8396	train-rmse:72.968
[8]	eval-rmse:94.0775	train-rmse:71.7214
[9]	eval-rmse:93.3743	train-rmse:70.4981
[10]	eval-rmse:92.7552	train-rmse:69.3315
[11]	eval-rmse:92.126	train-rmse:68.1706
[12]	eval-rmse:91.5266	train-rmse:66.9937
[13]	eval-rmse:90.9552	train-rmse:65.89
[14]	eval-rmse:90.3473	train-rmse:64.8099
[15]	eval-rmse:89.8431	train-rmse:63.7986
[16]	eval-rmse:89.3235	train-rmse:62.7585
[17]	eval-rmse:88.8282	train-rmse:61.7406
[18]	eval-rmse:88.3294	train-rmse:60.7693
[19]	eval-rmse:87.9069	train-rmse:59.7855
[20]	eval-rmse:87.4151	train-rmse:58.8153
[21]	eval-rmse:87.0153	train-rmse:57.8868
[22]	eval-rmse:86.6245	train-rmse:56.9407
[23]	eval-rmse:86.2682	train-rmse:56.0754
[24]	e

[194]	eval-rmse:85.633	train-rmse:8.86079
[195]	eval-rmse:85.6349	train-rmse:8.82982
[196]	eval-rmse:85.6551	train-rmse:8.78978
[197]	eval-rmse:85.6652	train-rmse:8.75792
[198]	eval-rmse:85.6863	train-rmse:8.71951
[199]	eval-rmse:85.7008	train-rmse:8.68001
[200]	eval-rmse:85.7219	train-rmse:8.64664
[201]	eval-rmse:85.7349	train-rmse:8.61222
[202]	eval-rmse:85.7484	train-rmse:8.58036
[203]	eval-rmse:85.7674	train-rmse:8.54616
[204]	eval-rmse:85.7829	train-rmse:8.51265
[205]	eval-rmse:85.8016	train-rmse:8.47964
[206]	eval-rmse:85.8203	train-rmse:8.45027
[207]	eval-rmse:85.8356	train-rmse:8.42069
[208]	eval-rmse:85.8516	train-rmse:8.39208
[209]	eval-rmse:85.8655	train-rmse:8.36272
[210]	eval-rmse:85.8769	train-rmse:8.33526
[211]	eval-rmse:85.8963	train-rmse:8.30624
[212]	eval-rmse:85.9111	train-rmse:8.27362
[213]	eval-rmse:85.9246	train-rmse:8.2482
[214]	eval-rmse:85.936	train-rmse:8.22274
[215]	eval-rmse:85.9475	train-rmse:8.19824
[216]	eval-rmse:85.9559	train-rmse:8.17277
[217]	eval-rms

KeyboardInterrupt: 

In [ ]:
data_path='./tap_fun_test.csv'
with open(data_path) as f:
    data_test=pd.read_csv(f)


In [ ]:
x=data_test.iloc[:,2:]
pca_x_preict=pca.transform(x)
dtest=xgb.DMatrix(pca_x_preict)
bst=xgb.Booster()
bst.load_model('0001.model')

y=bst.predict(dtest)

y[y<1]=0
retdf=pd.DataFrame({'user_id':data_test.iloc[:,0],
                    'prediction_pay_price':y})
retdf.to_csv('ret.csv',columns=['user_id','prediction_pay_price'],index=False)